# House Prices Competition:

## 1. Introdução:

<p>Neste projeto, você avaliará o desempenho e o poder preditivo de um modelo que foi treinado e testado em dados coletados em residências nos subúrbios de Boston, Massachusetts. Um modelo treinado sobre esses dados que é visto como um bom ajuste poderia então ser usado para fazer certas previsões sobre uma casa - em particular, seu valor monetário. Este modelo provaria ser inestimável para alguém como um agente imobiliário que poderia fazer uso de tais informações em uma base diária.</p>

Seguiremos estes passos para uma apresentação bem-sucedida do Concurso Kaggle:
- Carregando dos dados;
- Explorando os dados e Adequando os recursos e a variável de destino;
- Construindo um modelo;
- Fazendo e enviando previsões.

O objetivo da competição será lhe desafiar a <b>prever o preço final de cada casa.</b> 

## 2. Carregando os dados:

In [ ]:
'''
Importando Bibliotecas:
'''
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

'''
Carregamento de dados:
'''
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")


print("Dimensions of train:{}".format(train.shape))
print("Dimensions of test:{}".format(test.shape))

## 3. Explorando os dados e Adequando os recursos e a variável de destino:

In [ ]:
train.head()

In [ ]:
test.head()

Abaixo segue uma breve versão do que você encontrará no arquivo de descrição de dados:

- SalePrice - o preço de venda da propriedade em dólares; (<b>variável de destino</b>)
- MSSubClass - A classe de construção;
- MSZoning - A classificação geral de zoneamento;
- LotFrontage - Pés lineares de rua conectados à propriedade;
- LotArea - Tamanho do lote em pés quadrados;
- Street - Tipo de acesso rodoviário;
- Alley - Tipo de acesso ao beco;
- LotShape - forma geral da propriedade;
- LandContour - Planicidade da propriedade;
- Utilities - tipo de utilitários disponíveis;
- LotConfig - configuração de lotes.

Neste ponto, devemos começar a pensar sobre o que sabemos sobre os preços da habitação e o que poderíamos esperar ver neste conjunto de dados:

- Olhando para os dados, vemos os recursos esperados, como o YrSold (o ano em que a casa foi vendida pela última vez) e o SalePrice. 

- Outros que talvez não tivéssemos antecipado, como o LandSlope (a inclinação da terra onde a casa é construída) e RoofMatl (os materiais usados ​​para construir o telhado). 

- Mais tarde, teremos que tomar decisões sobre como abordaremos esses e outros recursos.

- Queremos fazer algumas plotagens durante o estágio de exploração de nosso projeto e precisaremos importar essa funcionalidade para o nosso ambiente também. 

- Plotagem nos permite visualizar a distribuição dos dados, verificar outliers e ver outros padrões que podemos perder de outra forma. 

- Usaremos o Matplotlib, uma biblioteca de visualização popular.

In [ ]:
train.SalePrice.describe()

<b>Informações estatísticas</b> do conjunto de dados train encontrados através do describe:

- A função <b>describe ()</b> lhe fornece informações estatísticas sobre o conjunto de dados analisado; 
- A função <b>count</b> exibe o número total de linhas do conjunto de dados; 
- Para dados numéricos, a função <b>describe ()</b> nos fornece os valores:
<p>1. medias;</p>
<p>2. desvios padrões;</p>
<p>3. quartis;</p>
<p>4. mínimos;</p>
<p>5. máximos.</p>
- O preço médio de venda de uma casa em nosso conjunto de dados é próximo a US $ 180.000;
- A maioria dos valores na faixa de <b>US 130.000(25 por cento)</b> a <b>US  215.000(75 por cento).</b>

Em seguida, verificaremos a <b>assimetria(skewness</b>), que é uma medida da forma da distribuição de valores:

- Ao executar a <b>regressão</b>, às vezes, faz sentido transformar a variável de destino quando está inclinada. Uma razão para isso é melhorar a <b>linearidade dos dados</b>;
- É importante ressaltar que as previsões geradas pelo modelo final também serão transformadas em log, portanto, precisaremos converter essas previsões novamente em sua forma original posteriormente;
- A função <b>np.log ()</b> irá transformar a variável, e <b>np.exp ()</b> irá reverter a transformação;
- Usamos a função <b>plt.hist ()</b> para traçar um histograma de SalePrice;
- Observe que a distribuição tem uma cauda mais longa à direita, sendo <b>positivamente inclinada</b>.

In [ ]:
print ("Skew is:", train.SalePrice.skew())
plt.hist(train.SalePrice, color='blue')
plt.title("SalesPrice - assimetria(skewness) ")
plt.show()

Agora usaremos a função  <b>np.log ()</b>: 
- Transformar train.SalePrice;
- Calcular a assimetria uma segunda vez;
- Re-plotar os dados;
- Um valor mais próximo de 0 significa que melhoramos a assimetria dos dados;
- Podemos ver visualmente que os dados se assemelham mais a uma distribuição normal.

In [ ]:
target = np.log(train.SalePrice)
print ("Skew is:", target.skew())
plt.title("SalesPrice - Log - assimetria(skewness) ")
plt.hist(target, color='blue')
plt.show()

### Trabalhando com recursos numéricos:

Agora que transformamos a variável de destino, vamos considerar nossos recursos:
- Primeiro, vamos verificar os recursos numéricos;
- Criar alguns gráficos;
- O <b>método .select_dtypes ()</b> retornará um subconjunto de colunas que correspondem aos tipos de dados especificados.

In [ ]:
numeric_features = train.select_dtypes(include=[np.number])
numeric_features.dtypes

<p>O <b>método DataFrame.corr ()</b> exibe a correlação (ou relacionamento) entre as colunas.</p> 
<p>Examinaremos as <b>correlações</b> entre os <b>recursos</b> e a <b>variável de destino</b>.</p>

In [ ]:
corr = numeric_features.corr()

print (corr['SalePrice'].sort_values(ascending=False)[:5], '\n')
print (corr['SalePrice'].sort_values(ascending=False)[-5:])

Os cinco primeiros recursos são os mais positivamente correlacionados com o SalePrice, enquanto os próximos cinco são os mais correlacionados negativamente.

Vamos nos aprofundar no OverallQual. Podemos usar o método .unique () para obter os valores exclusivos.

In [ ]:
train.OverallQual.unique()

- Os dados de <b>OverallQual</b> são valores inteiros no intervalo de 1 a 10, inclusive;
- Podemos criar uma tabela dinâmica para investigar melhor a relação entre <b>OverallQual</b> e <b>SalePrice</b>;
- Os documentos do Pandas demonstram como realizar essa tarefa.;
- Nós definimos <b>index = 'OverallQual</b>' e <b>values = 'SalePrice'</b>. 
- Nós escolhemos olhar para a <b>mediana.</b>

In [ ]:
quality_pivot = train.pivot_table(index='OverallQual',values='SalePrice', aggfunc=np.median)
quality_pivot

- Para nos ajudar a visualizar essa tabela dinâmica mais facilmente, podemos criar uma plotagem de barras usando o <b>método Series.plot ().</b>

In [ ]:
quality_pivot.plot(kind='bar', color='blue')
plt.xlabel('Overall Quality')
plt.ylabel('Median Sale Price')
plt.xticks(rotation=0)
plt.show()

- Observe que o preço mediano de vendas aumenta estritamente à medida que a Qualidade geral aumenta;
- Em seguida, vamos usar o <b>plt.scatter ()</b> para gerar alguns gráficos de dispersão e visualizar a relação entre a área de vida do solo <b>GrLivArea</b> e o <b>SalePrice.</b>

In [ ]:
plt.scatter(x=train['GrLivArea'], y=target)
plt.ylabel('Sale Price')
plt.xlabel('Above grade (ground) living area square feet')
plt.show()

- À primeira vista, vemos que os aumentos na área habitacional correspondem a aumentos de preço;
- Nós faremos o mesmo para o GarageArea.

In [ ]:
plt.scatter(x=train['GarageArea'], y=target)
plt.ylabel('Sale Price')
plt.xlabel('Garage Area')
plt.show()

- Observe que há muitas casas com 0 para a área de garagem, indicando que elas não têm garagem;
- Vamos transformar outros recursos posteriormente para refletir essa suposição;
- Existem alguns outliers também. Os outliers podem afetar um modelo de regressão, afastando nossa linha de regressão estimada da verdadeira linha de regressão da população. Então, vamos remover essas observações dos nossos dados.;
- Remover os outliers é uma arte e uma ciência. Existem muitas técnicas para lidar com outliers.

Vamos criar um novo dataframe com alguns outliers removidos.

In [ ]:
train = train[train['GarageArea'] < 1200]

- Vamos dar outra olhada:

In [ ]:
plt.scatter(x=train['GarageArea'], y=np.log(train.SalePrice))
plt.xlim(-200,1600) # This forces the same scale as before
plt.ylabel('Sale Price')
plt.xlabel('Garage Area')
plt.show()

### Manipulando valores nulos:

- Em seguida, examinaremos os valores nulos ou ausentes;
- Vamos criar um DataFrame para visualizar as principais colunas nulas;
- Encadeando os métodos <b>train.isnull (). Sum ()</b>, retornamos uma série das contagens dos valores nulos em cada coluna.

In [ ]:
nulls = pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

- A documentação pode nos ajudar a entender os valores ausentes;
- No caso do <b>PoolQC</b>, a coluna se refere à <b>qualidade da piscina</b>. A qualidade da piscina é NaN quando PoolArea é 0, ou não há piscina;
- Podemos encontrar um relacionamento semelhante entre muitas das colunas relacionadas à <b>Garagem</b>;
- Vamos dar uma olhada em uma das outras colunas, <b>MiscFeature</b>;
- Usaremos o <b>método Series.unique ()</b> para retornar uma lista dos valores exclusivos.

In [ ]:
print ("Unique values are:", train.MiscFeature.unique())

- Podemos usar a documentação para descobrir o que esses valores indicam:
   - Elev = Elevador;
   - Gar2 = Garagem do segundo piso (se não for descrito na seção da garagem);
   - Othr = Outos;
   - Shed = Galpão (mais de 100 SF);
   - TenC = Quadra de Tênis;
   - NA =  Nada.
- Esses valores descrevem se a casa tem ou não um galpão com mais de 100 pés quadrados;
- Uma segunda garagem e assim por diante. 
- Podemos querer usar essa informação mais tarde; 
- É importante reunir conhecimento de domínio para tomar as melhores decisões ao lidar com dados ausentes.

### Limpando os recursos não numéricos:

In [ ]:
categoricals = train.select_dtypes(exclude=[np.number])
categoricals.describe()

- A coluna <b>count</b> indica a contagem de observações não nulas;
- Enquanto a <b>unique</b> conta o número de valores únicos;
- <b>top</b> é o valor mais comum, com a frequência do valor máximo mostrado por <b>freq</b>;
- Para muitos desses recursos, podemos querer usar uma codificação quente para fazer uso das informações para modelagem;
- Uma codificação quente é uma técnica que irá transformar dados categóricos em números para que o modelo possa entender se uma observação particular cai ou não em uma categoria ou outra.

### Transformação e engenharia de resursos:

- Ao transformar recursos, é importante lembrar que quaisquer transformações aplicadas aos dados de treinamento antes de ajustar o modelo devem ser aplicadas aos dados de teste;
- Nosso modelo espera que a forma dos recursos do conjunto de trens corresponda àqueles do conjunto de testes; 
- Isso significa que qualquer engenharia de recursos que tenha ocorrido durante o trabalho nos dados do trem deve ser aplicada novamente no conjunto de testes;
- Para demonstrar como isso funciona, considere os dados do Street, que indicam se há acesso por estrada de cascalho ou pavimentada à propriedade.

In [ ]:
print ("Original: \n") 
print (train.Street.value_counts(), "\n")

- Na coluna <b>Street</b>, os valores exclusivos são <b>Pave</b> e <b>Grvl</b>, que descrevem o tipo de acesso rodoviário à propriedade;
- No conjunto de <b>train</b>, apenas 5 residências possuem acesso de cascalho;
- Nosso modelo precisa de dados numéricos, portanto, usaremos uma <b>codificação quente</b> para transformar os dados em uma coluna booleana;
- Criamos uma nova coluna chamada <b>enc_street</b>;
- O método <b>pd.get_dummies ()</b> manipulará isso para nós;
- Como mencionado anteriormente, precisamos fazer isso nos dados do <b>train</b> e do <b>test</b>.

In [ ]:
train['enc_street'] = pd.get_dummies(train.Street, drop_first=True)
test['enc_street'] = pd.get_dummies(train.Street, drop_first=True)

In [ ]:
print ('Encoded: \n') 
print (train.enc_street.value_counts())

- Os valores concordam;
- Nós projetamos nosso primeiro recurso;
- Engenharia de recursos é o processo de fazer recursos dos dados adequados para uso em modelagem e aprendizado de máquina.;
- Quando codificamos o recurso <b>Street</b> em uma coluna de valores booleanos, projetamos um recurso;
- Vamos tentar criar outro recurso;
- Veremos o <b>SaleCondition</b> construindo e plotando uma tabela dinâmica, como fizemos anteriormente para o <b>OverallQual</b>.

In [ ]:
condition_pivot = train.pivot_table(index='SaleCondition',
                                    values='SalePrice', aggfunc=np.median)
condition_pivot.plot(kind='bar', color='blue')
plt.xlabel('Sale Condition')
plt.ylabel('Median Sale Price')
plt.xticks(rotation=0)
plt.show()

<b>Observações:</b>
- Observe que o Partial tem um Preço de Venda Mediano significativamente maior do que os outros;
- Vamos codificar isso como um novo recurso;
- Selecionamos todas as casas onde SaleCondition é igual a Patrial e atribuímos o valor 1, caso contrário, atribuir 0.
- Abaixo um método semelhante ao usado na Street acima.

In [ ]:
def encode(x): return 1 if x == 'Partial' else 0
train['enc_condition'] = train.SaleCondition.apply(encode)
test['enc_condition'] = test.SaleCondition.apply(encode)

<b>Observações:</b>
- Vamos explorar esse novo recurso como um gráfico.

In [ ]:
condition_pivot = train.pivot_table(index='enc_condition', values='SalePrice', aggfunc=np.median)
condition_pivot.plot(kind='bar', color='blue')
plt.xlabel('Encoded Sale Condition')
plt.ylabel('Median Sale Price')
plt.xticks(rotation=0)
plt.show()

<b>Observações:</b>
- Isso parece ótimo;
- Você pode continuar trabalhando com mais recursos para melhorar o desempenho máximo do seu modelo;
- Antes de preparar os dados para modelagem, precisamos lidar com os dados ausentes;
- Vamos preencher os valores ausentes com um valor médio e, em seguida, atribuir os resultados aos dados;
- Este é um método de interpolação. O método DataFrame.interpolate () simplifica isso.
- Esse é um método rápido e simples de lidar com valores ausentes e pode não levar ao melhor desempenho do modelo em novos dados. 
- O manuseio de valores ausentes é uma parte importante do processo de modelagem, onde a criatividade e a percepção podem fazer uma grande diferença. 
- Esta é outra área onde você pode estender este tutorial

In [ ]:
data = train.select_dtypes(include=[np.number]).interpolate().dropna()

<b>Observações:</b>
Verifique se todas as colunas têm 0 valores nulos.

In [ ]:
sum(data.isnull().sum() != 0)

## 4. Fazendo e enviando previsões:

<b>Vamos executar as etapas finais para preparar nossos dados para modelagem:</b>
- Vamos separar os recursos e a variável de destino para modelagem. 
- Nós atribuiremos os recursos a X e a variável de destino a y. 
- Usamos np.log () como explicado acima para transformar a variável y para o modelo. data.drop ([features], axis = 1) diz aos pandas quais colunas queremos excluir. 
- Não incluiremos SalePrice por motivos óbvios, e Id é apenas um índice sem relação com o SalePrice.

In [ ]:
y = np.log(train.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)

<b>Vamos particionar os dados e começar a modelagem:</b>
- Usaremos a função train_test_split () do scikit-learn para criar um conjunto de treinamento e um conjunto de hold-out;
- Particionar os dados dessa maneira nos permite avaliar como nosso modelo pode funcionar em dados que nunca foi visto antes;
- Se nós treinarmos o modelo em todos os dados de teste, será difícil dizer se ocorreu um overfitting;

<b>train_test_split () retorna quatro objetos:</b>
- X_train é o subconjunto de nossos recursos usados ​​para treinamento;
- X_test é o subconjunto que será o nosso conjunto de 'hold-out' - o que vamos usar para testar o modelo;
- y_train é a variável alvo SalePrice que corresponde a X_train;
- y_test é a variável de destino SalePrice, que corresponde ao X_test;
- O primeiro valor de parâmetro X denota o conjunto de dados do preditor e y é a variável de destino;
- Em seguida, definimos random_state = 42. Isso fornece resultados reproduzíveis, pois o train_test_split do sci-kit learn particionará os dados aleatoriamente;
- O parâmetro test_size informa à função que proporção dos dados deve estar na partição de teste. Neste exemplo, cerca de 33% dos dados são dedicados ao conjunto de holdout.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, random_state=42, test_size=.33)

### Começando a desenvolver o Algoritmo:

<b>Vamos primeiro criar um modelo de regressão linear:</b>
- Primeiro, instanciamos o modelo.

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()

<b>Em seguida, precisamos ajustar o modelo:</b>
- Primeiro instanciar o modelo e depois ajustar o modelo;
- O ajuste de modelo é um procedimento que varia para diferentes tipos de modelos;
- Simplificando, estamos estimando a relação entre nossos preditores e a variável de destino para que possamos fazer previsões precisas sobre novos dados;
- Nós ajustamos o modelo usando X_train e y_train, e marcaremos com X_test e y_test.;
- O método lr.fit () ajustará a regressão linear nos recursos e na variável de destino que nós passamos.

In [ ]:
model = lr.fit(X_train, y_train)

### Avalie o desempenho e visualize os resultados:

<b>Agora, queremos avaliar o desempenho do modelo:</b>
- Cada competição pode avaliar as submissões de maneira diferente;
- Nesta competição, a Kaggle avaliará nosso envio usando o RMSE (root mean mean squared error);
- Também veremos o valor de r ao quadrado;
- O valor de r-quadrado é uma medida de quão próximos os dados estão da linha de regressão ajustada;
- É preciso um valor entre 0 e 1, 1, o que significa que toda a variação no alvo é explicada pelos dados;
- Em geral, um maior valor de r-quadrado significa um melhor ajuste;
- O método model.score () retorna o valor de r ao quadrado por padrão.

In [ ]:
print ("R^2 is: \n", model.score(X_test, y_test))

<b>Observações:</b>
- Isso significa que nossos recursos explicam aproximadamente 89% da variação em nossa variável de destino;
- Em seguida, vamos considerar <b>rmse</b>;
- Para fazer isso, use o modelo que criamos para fazer previsões no conjunto de dados de teste.

In [ ]:
predictions = model.predict(X_test)

- O método <b>model.predict ()</b> retornará uma lista de previsões dadas a um conjunto de preditores;
- Use <b>model.predict ()</b> depois de ajustar o modelo;
- A função <b>mean_squared_error</b> usa dois arrays e calcula o <b>rmse.</b>

In [ ]:
from sklearn.metrics import mean_squared_error
print ('RMSE is: \n', mean_squared_error(y_test, predictions))

<b>Observações:</b>
- Interpretar este valor é um pouco mais intuitivo que o valor de r ao quadrado;
- O RMSE mede a distância entre nossos valores previstos e valores reais;
- Podemos ver essa relação graficamente com um gráfico de dispersão.

In [ ]:
actual_values = y_test
plt.scatter(predictions, actual_values, alpha=.75,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Linear Regression Model')
plt.show()

<b>Observações:</b>
- Se nossos valores previstos fossem idênticos aos valores reais, esse gráfico seria a linha reta y = x porque cada valor previsto x seria igual a cada valor real y.

### Tente melhorar o Algoritmo:

<b>Em seguida, tentaremos usar o Regularização de cume para diminuir a influência de recursos menos importantes:</b>
- A regularização de cume é um processo que reduz os coeficientes de regressão de características menos importantes;
- Mais uma vez, vamos instanciar o modelo;
- O modelo de regularização de Ridge usa um parâmetro alpha, que controla a força da regularização;
- Vamos experimentar percorrendo alguns valores diferentes de alfa e ver como isso altera nossos resultados.

In [ ]:
for i in range (-2, 3):
    alpha = 10**i
    rm = linear_model.Ridge(alpha=alpha)
    ridge_model = rm.fit(X_train, y_train)
    preds_ridge = ridge_model.predict(X_test)

    plt.scatter(preds_ridge, actual_values, alpha=.75, color='b')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.title('Ridge Regularization with alpha = {}'.format(alpha))
    overlay = 'R^2 is: {}\nRMSE is: {}'.format(
                    ridge_model.score(X_test, y_test),
                    mean_squared_error(y_test, preds_ridge))
    plt.annotate(s=overlay,xy=(12.1,10.6),size='x-large')
    plt.show()

<b>Observações:</b>
- Esses modelos são quase idênticos ao primeiro modelo;
- No nosso caso, ajustar o alfa não melhorou substancialmente nosso modelo;
- À medida que você adiciona mais recursos, a regularização pode ser útil;
- Repita este passo depois de adicionar mais funcionalidades.

## 5. Processando o envio do algoritmo:

<b>Precisamos criar um csv que contenha o SalePrice previsto para cada observação no conjunto de dados test.csv:</b>
- Entraremos em nossa conta do Kaggle e acessaremos a página de envio para fazer um envio;
- Nós usaremos o DataFrame.to_csv () para criar um csv para enviar;
- A primeira coluna deve conter o ID dos dados de teste.

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test.Id

<b>Agora, selecione os recursos dos dados de teste para o modelo, como fizemos anteriormente:</b>

In [ ]:
feats = test.select_dtypes(
        include=[np.number]).drop(['Id'], axis=1).interpolate()

<b>Em seguida, geramos nossas previsões:</b>

In [ ]:
predictions = model.predict(feats)

<b>Agora vamos transformar as previsões para o formulário correto:</b>
- Lembre-se que para inverter <b>log () fazemos exp ();</b>
- Então, aplicaremos <b>np.exp ()</b> às nossas predições, porque tomamos o logaritmo anteriormente.

In [ ]:
final_predictions = np.exp(predictions)

<b>Olhe a diferença:</b>.

In [ ]:
print ("Original predictions are: \n", predictions[:5], "\n")
print ("Final predictions are: \n", final_predictions[:5])

<b>Vamos atribuir essas previsões e verificar se tudo parece bem:</b>

In [ ]:
submission['SalePrice'] = final_predictions
submission.head()

<b>Observações:</b>
- Estamos confiantes de que temos os dados organizados no formato adequado, podemos exportar para um arquivo .csv como Kaggle espera;
- Nós passamos de index = False porque os Pandas, caso contrário, criariam um novo índice para nós.

### Envie nossos resultados:

<b>Observações:</b>
- Criamos um arquivo chamado submission1.csv em nosso diretório de trabalho que está em conformidade com o formato correto;
- Vá para a página de envio para fazer um envio.

In [ ]:
submission.to_csv('submission1.csv', index=False)

### Próximos passos:

<b>Observações:</b>
- Você pode estender este trabalho e melhorar seus resultados;
- Trabalhando com e transformando outros recursos no conjunto de treinamento;
- Experiências com diferentes técnicas de modelagem, como Regressores Florestais Aleatórios ou Intensificação de Gradiente;

<b>Usando modelos conjuntos:</b>
- Criamos um conjunto de recursos categóricos chamados categoricals que não foram incluídos no modelo final;
- Volte e tente incluir esses recursos;
- Existem outros métodos que podem ajudar com dados categóricos, principalmente o método pd.get_dummies ();
- Depois de trabalhar nesses recursos, repita as transformações para os dados de teste e faça outra submissão.
- Trabalhar com modelos e participar de competições de Kaggle pode ser um processo interativo - é importante experimentar novas ideias, aprender sobre os dados e testar novos modelos e técnicas.
- Com essas ferramentas, você pode aproveitar seu trabalho e melhorar seus resultados.